<a href="https://colab.research.google.com/github/JIYOON-LEE-practice/SeoulBike/blob/main/SeoulBike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 서울 자전거 대여 예측
github csv url: https://raw.githubusercontent.com/JIYOON-LEE-practice/SeoulBike/main/SeoulBikeData(edit_dayoftheweek_erasedate).csv

In [46]:
# 라이브러리 사용
import tensorflow as tf
import pandas as pd

In [47]:
# 1. 과거의 서울 자전거 대여 데이터 준비(요일 표시 처리함. 날짜 제외시킴.)
파일경로 = 'https://raw.githubusercontent.com/JIYOON-LEE-practice/SeoulBike/main/SeoulBikeData(edit_dayoftheweek_erasedate).csv'
#파일 경로는 raw데이터로 추출해야 함. private프로젝트로 설정되어 있으면 데이터의 token이 계속 바뀌어 오류남. public으로 해야.
서울자전거 = pd.read_csv(파일경로, encoding='cp949') #온도표시가 포함되어 있다고 오류나서 encoding='cp949' 붙여줌, 캜로 바뀌어 나옴ㅜ
print(서울자전거.columns)
서울자전거.head()

Index(['Rented Bike Count', 'Hour', 'Temperature(캜)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(캜)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
       'Holiday', 'Functioning Day', 'Day of the week'],
      dtype='object')


,Rented Bike Count,Hour,Temperature(캜),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(캜),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,Day of the week
0,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,Friday
1,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,Friday
2,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes,Friday
3,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,Friday
4,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes,Friday


In [48]:
# 원핫인코딩
서울자전거 = pd.get_dummies(서울자전거)
print(서울자전거.columns)
서울자전거.head()

Index(['Rented Bike Count', 'Hour', 'Temperature(캜)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(캜)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)',
       'Seasons_Autumn', 'Seasons_Spring', 'Seasons_Summer', 'Seasons_Winter',
       'Holiday_Holiday', 'Holiday_No Holiday', 'Functioning Day_No',
       'Functioning Day_Yes', 'Day of the week_Friday',
       'Day of the week_Monday', 'Day of the week_Saturday',
       'Day of the week_Sunday', 'Day of the week_Thursday',
       'Day of the week_Tuesday', 'Day of the week_Wednesday'],
      dtype='object')


,Rented Bike Count,Hour,Temperature(캜),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(캜),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons_Autumn,Seasons_Spring,Seasons_Summer,Seasons_Winter,Holiday_Holiday,Holiday_No Holiday,Functioning Day_No,Functioning Day_Yes,Day of the week_Friday,Day of the week_Monday,Day of the week_Saturday,Day of the week_Sunday,Day of the week_Thursday,Day of the week_Tuesday,Day of the week_Wednesday
0,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0
1,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0
2,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0
3,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0
4,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0


In [49]:
# 독립변수, 종속변수 분리 
독립 = 서울자전거[['Hour', 'Temperature(캜)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(캜)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)',
       'Seasons_Autumn', 'Seasons_Spring', 'Seasons_Summer', 'Seasons_Winter',
       'Holiday_Holiday', 'Holiday_No Holiday', 'Functioning Day_No',
       'Functioning Day_Yes', 'Day of the week_Friday',
       'Day of the week_Monday', 'Day of the week_Saturday',
       'Day of the week_Sunday', 'Day of the week_Thursday',
       'Day of the week_Tuesday', 'Day of the week_Wednesday']]
종속 = 서울자전거[['Rented Bike Count']]
print(독립.shape, 종속.shape)

(8760, 24) (8760, 1)


In [50]:
# 2. 모델의 구조 만들기
X = tf.keras.layers.Input(shape=[24])
H = tf.keras.layers.Dense(10, activation="swish")(X)
H = tf.keras.layers.Dense(10, activation="swish")(H)
H = tf.keras.layers.Dense(10, activation="swish")(H)
H = tf.keras.layers.Dense(10, activation="swish")(H)
H = tf.keras.layers.Dense(10, activation="swish")(H)
H = tf.keras.layers.Dense(10, activation="swish")(H)
Y = tf.keras.layers.Dense(1)(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='mse')

In [51]:
# 3.데이터로 모델을 학습(FIT)합니다.
model.fit(독립, 종속, epochs=1000, verbose=0)
model.fit(독립, 종속, epochs=10)

Epoch 1/10
274/274 [==============================] - 1s 2ms/step - loss: 42205.1484
Epoch 2/10
274/274 [==============================] - 1s 2ms/step - loss: 51867.8594
Epoch 3/10
274/274 [==============================] - 1s 2ms/step - loss: 42086.2500
Epoch 4/10
274/274 [==============================] - 0s 2ms/step - loss: 42576.3750
Epoch 5/10
274/274 [==============================] - 0s 2ms/step - loss: 41360.8047
Epoch 6/10
274/274 [==============================] - 0s 2ms/step - loss: 42321.3555
Epoch 7/10
274/274 [==============================] - 0s 2ms/step - loss: 42368.6797
Epoch 8/10
274/274 [==============================] - 0s 2ms/step - loss: 42064.8477
Epoch 9/10
274/274 [==============================] - 0s 2ms/step - loss: 41665.8125
Epoch 10/10
274/274 [==============================] - 0s 2ms/step - loss: 42739.6914


In [52]:
# 4. 모델을 이용합니다
print(model.predict(독립[:10]))
# 종속변수 확인
print(종속[:10])

[[164.96634 ]
 [131.90726 ]
 [106.709526]
 [ 88.44295 ]
 [ 70.86643 ]
 [ 67.036194]
 [ 85.190865]
 [133.48253 ]
 [504.8166  ]
 [336.7851  ]]
   Rented Bike Count
0                254
1                204
2                173
3                107
4                 78
5                100
6                181
7                460
8                930
9                490


In [53]:
# 모델의 수식 확인
print(model.get_weights())

[array([[-5.8855131e-02, -3.8096633e-02,  2.7702367e+00,  7.1283336e+00,
         1.1689043e+00, -1.0316774e-01,  1.5590506e-02, -1.8800310e+00,
        -6.5383188e-02, -1.8415196e+00],
       [ 3.3140185e-01, -1.9510232e-02,  2.1284227e-01, -9.9398321e-01,
         1.1793797e+00,  9.1006166e-01, -6.8065532e-02,  1.9815564e+00,
         2.6207164e-01, -8.4075814e-01],
       [-2.4186276e-01, -4.9418598e-02, -8.4260648e-01,  1.1019499e-01,
         4.4520251e-02,  7.0884061e-01, -1.2067304e-02,  5.7081300e-01,
        -2.6335952e-01,  7.0505011e-01],
       [-2.3158678e-01, -9.9686116e-02, -3.3501029e-01,  4.9042478e-02,
        -1.8237056e-01,  6.3929975e-02,  1.8499711e-01, -1.2949839e-01,
         1.3854118e-01,  1.5003628e+00],
       [-3.8416579e-01, -2.6430939e-03,  2.4143888e-02,  2.8854650e-02,
         1.6749533e-02,  5.2732695e-02, -5.2037076e-03,  2.0808529e-02,
        -2.9710895e-01,  4.5533299e-02],
       [-2.8294539e-01,  1.6932098e-02,  2.3519433e+00,  5.8960509e-01,
  